In [ ]:
BRANCH = 'main'
# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-7jr_v61s/nemo-toolkit_89a68e327392484b9098dd27ce4ac003
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-7jr_v61s/nemo-toolkit_89a68e327392484b9098dd27ce4ac003
     |████████████████████████████████| 12.7 MB 4.1 MB/s 
     |████████████████████████████████| 109 kB 49.7 MB/s 
     |████████████████████████████████| 1.2 MB 44.4 MB/s 
     |████████████████████████████████| 235 kB 46.0 MB/s 
     |████████████████████████████████| 131 kB 48.1 MB/s 
     |████████████████████████████████| 11.2 MB 37.2 MB/s 
     |████████████████████████████████| 1.7 MB 45.2 MB/s 
     |████████████████████████████████| 893 kB 45.7 MB/s 
     |████████████████████████████████| 90 kB 9.8 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 1.5 MB 43.5 MB/s 
     |████████████████████████████████| 68 kB 7.0 MB/s 
     |████████████████████████

In [ ]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

[NeMo W 2022-01-01 07:00:08 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-01-01 07:00:11 __init__:23] `pynini` is not installed ! 
    Please run the `nemo_text_processing/setup.sh` scriptprior to usage of this toolkit.


# DOWNLOAD SQuAD v1.1 DATASET

In [ ]:
# set the following paths
DATA_DIR = "PATH_TO_DATA"
WORK_DIR = "PATH_TO_CHECKPOINTS_AND_LOGS"

## download get_squad.py script to download and preprocess the SQuAD data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_squad.py'):
    print('Downloading get_squad.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/get_squad.py', WORK_DIR)
else:
    print ('get_squad.py already exists')

In [ ]:
# download and preprocess the data
! python $WORK_DIR/get_squad.py --destDir $DATA_DIR

[NeMo I 2022-01-01 07:01:40 get_squad:66] PATH_TO_DATA
[NeMo I 2022-01-01 07:01:40 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
[NeMo I 2022-01-01 07:01:42 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
[NeMo I 2022-01-01 07:01:44 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
[NeMo I 2022-01-01 07:01:49 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json


In [ ]:
! ls -LR {DATA_DIR}/squad

PATH_TO_DATA/squad:
v1.1  v2.0

PATH_TO_DATA/squad/v1.1:
dev-v1.1.json  train-v1.1.json

PATH_TO_DATA/squad/v2.0:
dev-v2.0.json  train-v2.0.json


In [ ]:
import json
 
# Opening JSON file
f = open('/content/PATH_TO_DATA/squad/v1.1/dev-v1.1.json')
 
data = json.load(f)
 
print(json.dumps(data["data"][0], indent=4))
 
# Closing file
f.close()

{
    "title": "Super_Bowl_50",
    "paragraphs": [
        {
            "context": "Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the \"golden anniversary\" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as \"Super Bowl L\"), so that the logo could prominently feature the Arabic numerals 50.",
            "qas": [
                {
                    "answers": [
                        {
                            "

In [ ]:
# This is the model configuration file that we will download, do not change this
MODEL_CONFIG = "question_answering_squad_config.yaml"

# model parameters, play with these
BATCH_SIZE = 12
MAX_SEQ_LENGTH = 384
# specify BERT-like model, you want to use
PRETRAINED_BERT_MODEL = "bert-base-uncased"
TOKENIZER_NAME = "bert-base-uncased" # tokenizer name

# Number of data examples used for training, validation, test and inference
TRAIN_NUM_SAMPLES = VAL_NUM_SAMPLES = TEST_NUM_SAMPLES = 10000 
INFER_NUM_SAMPLES = 5 # EXAMPLE

TRAIN_FILE = f"{DATA_DIR}/squad/v1.1/train-v1.1.json"
VAL_FILE = f"{DATA_DIR}/squad/v1.1/dev-v1.1.json"
TEST_FILE = f"{DATA_DIR}/squad/v1.1/dev-v1.1.json"
INFER_FILE = f"{DATA_DIR}/squad/v1.1/dev-v1.1.json"

INFER_PREDICTION_OUTPUT_FILE = "output_prediction.json"
INFER_NBEST_OUTPUT_FILE = "output_nbest.json"

# training parameters
LEARNING_RATE = 0.00003

# number of epochs
MAX_EPOCHS = 4

In [ ]:
# download the model's default configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/conf/{MODEL_CONFIG}', config_dir)
else:
    print ('config file is already exists')

# MODEL CONFIG

In [ ]:
# this line will print the entire default config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

PATH_TO_CHECKPOINTS_AND_LOGS/configs/question_answering_squad_config.yaml
name: QA
pretrained_model: null
do_training: true
trainer:
  gpus: 1
  num_nodes: 1
  max_epochs: 2
  max_steps: null
  accumulate_grad_batches: 1
  precision: 16
  accelerator: ddp
  gradient_clip_val: 0.0
  val_check_interval: 1.0
  checkpoint_callback: false
  logger: false
  num_sanity_val_steps: 0
  log_every_n_steps: 1
model:
  nemo_path: null
  dataset:
    version_2_with_negative: false
    doc_stride: 128
    max_query_length: 64
    max_seq_length: 384
    max_answer_length: 30
    null_score_diff_threshold: 0.0
    n_best_size: 20
    use_cache: true
    do_lower_case: true
    num_workers: 2
    pin_memory: false
    drop_last: false
  train_ds:
    file: null
    batch_size: 24
    shuffle: true
    num_samples: -1
    num_workers: ${model.dataset.num_workers}
    drop_last: ${model.dataset.drop_last}
    pin_memory: ${model.dataset.pin_memory}
  validation_ds:
    file: null
    batch_size: 24
    s

In [ ]:
config.model.train_ds.file = TRAIN_FILE
config.model.validation_ds.file = VAL_FILE
config.model.test_ds.file = TEST_FILE

config.model.train_ds.num_samples = TRAIN_NUM_SAMPLES
config.model.validation_ds.num_samples = VAL_NUM_SAMPLES
config.model.test_ds.num_samples = TEST_NUM_SAMPLES

config.model.tokenizer.tokenizer_name = TOKENIZER_NAME

# CONFIG MODEL AND TRAINER THEO MONG MUỐN

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

config.trainer.max_epochs = MAX_EPOCHS

# Remove distributed training flags if only running on a single GPU or CPU
config.trainer.accelerator = None

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

trainer = pl.Trainer(**config.trainer)

Using 16bit native Automatic Mixed Precision (AMP)
[NeMo W 2022-01-01 07:16:40 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:49: LightningDeprecationWarning: Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7. Use `max_steps = -1` instead.
      "Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7."
    
[NeMo W 2022-01-01 07:16:40 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
      f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Trainer config - 

gpus: 1
num_nodes: 1
max_epochs: 4
max_steps: null
accumulate_grad_batches: 1
precision: 16
accelerator: null
gradient_clip_val: 0.0
val_check_interval: 1.0
checkpoint_callback: false
logger: false
num_sanity_val_steps: 0
log_every_n_steps: 1



In [ ]:
config.exp_manager.exp_dir = WORK_DIR
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)

[NeMo I 2022-01-01 07:17:01 exp_manager:283] Experiments will be logged at PATH_TO_CHECKPOINTS_AND_LOGS/QA/2022-01-01_07-17-01
[NeMo I 2022-01-01 07:17:01 exp_manager:648] TensorboardLogger has been set up


[NeMo W 2022-01-01 07:17:01 exp_manager:880] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.
[NeMo W 2022-01-01 07:17:01 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:244: LightningDeprecationWarning: `ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6. Please use `every_n_epochs` instead.
      "`ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6."
    


In [ ]:
# complete list of supported BERT-like models
nemo_nlp.modules.get_pretrained_lm_models_list()

['bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'bert-base-german-dbmdz-cased',
 'bert-base-german-dbmdz-uncased',
 'cl-tohoku/bert-base-japanese',
 'cl-tohoku/bert-base-japanese-whole-word-masking',
 'cl-tohoku/bert-base-japanese-char',
 'cl-tohoku/bert-base-japanese-char-whole-word-masking',
 'TurkuNLP/bert-base-finnish-cased-v1',
 'TurkuNLP/bert-base-finnish-uncased-v1',
 'wietsedv/bert-base-dutch-cased',
 'distilbert-base-uncased',
 'distilbert-base-uncased-distilled-squad',
 'distilbert-base-cased',
 'distilbert-base-cased-distilled-squad',
 'distilbert-base-german-cased',
 'distilbert-base-multilingu

In [ ]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL
config.model.train_ds.batch_size = BATCH_SIZE
config.model.validation_ds.batch_size  = BATCH_SIZE
config.model.test_ds.batch_size = BATCH_SIZE
config.model.optim.lr = LEARNING_RATE

print("Updated model config - \n")
print(OmegaConf.to_yaml(config.model))

Updated model config - 

nemo_path: null
dataset:
  version_2_with_negative: false
  doc_stride: 128
  max_query_length: 64
  max_seq_length: 384
  max_answer_length: 30
  null_score_diff_threshold: 0.0
  n_best_size: 20
  use_cache: true
  do_lower_case: true
  num_workers: 2
  pin_memory: false
  drop_last: false
train_ds:
  file: PATH_TO_DATA/squad/v1.1/train-v1.1.json
  batch_size: 12
  shuffle: true
  num_samples: 10000
  num_workers: ${model.dataset.num_workers}
  drop_last: ${model.dataset.drop_last}
  pin_memory: ${model.dataset.pin_memory}
validation_ds:
  file: PATH_TO_DATA/squad/v1.1/dev-v1.1.json
  batch_size: 12
  shuffle: false
  num_samples: 10000
  num_workers: ${model.dataset.num_workers}
  drop_last: ${model.dataset.drop_last}
  pin_memory: ${model.dataset.pin_memory}
test_ds:
  file: PATH_TO_DATA/squad/v1.1/dev-v1.1.json
  batch_size: 12
  shuffle: false
  num_samples: 10000
  num_workers: ${model.dataset.num_workers}
  drop_last: ${model.dataset.drop_last}
  pin_mem

# PROCESSING DATA PREPARED FOR TRAINING And VALIDATION

In [ ]:
# initialize the model
# dataset we'll be prepared for training and evaluation during
model = nemo_nlp.models.QAModel(cfg=config.model, trainer=trainer)

[NeMo I 2022-01-01 07:21:15 tokenizer_utils:125] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
100%|██████████| 442/442 [00:42<00:00, 10.47it/s]


[NeMo I 2022-01-01 07:22:07 qa_dataset:123] Preprocessing data.
[NeMo I 2022-01-01 07:22:07 qa_squad_processing:558] *** Example ***
[NeMo I 2022-01-01 07:22:07 qa_squad_processing:559] unique_id: 1000000000
[NeMo I 2022-01-01 07:22:07 qa_squad_processing:560] example_index: 0
[NeMo I 2022-01-01 07:22:07 qa_squad_processing:561] doc_span_index: 0
[NeMo I 2022-01-01 07:22:07 qa_squad_processing:562] tokens: [CLS] to whom did the virgin mary allegedly appear in 1858 in lou ##rdes france ? [SEP] architectural ##ly , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms up ##rai ##sed with the legend " ve ##ni ##te ad me om ##nes " . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the gr ##otto , a marian place of prayer and reflection . it is a replica of the gr ##otto at lou ##rdes 

100%|██████████| 48/48 [00:05<00:00,  9.09it/s]


[NeMo I 2022-01-01 07:24:17 qa_dataset:123] Preprocessing data.
[NeMo I 2022-01-01 07:24:17 qa_squad_processing:558] *** Example ***
[NeMo I 2022-01-01 07:24:17 qa_squad_processing:559] unique_id: 1000000000
[NeMo I 2022-01-01 07:24:17 qa_squad_processing:560] example_index: 0
[NeMo I 2022-01-01 07:24:17 qa_squad_processing:561] doc_span_index: 0
[NeMo I 2022-01-01 07:24:17 qa_squad_processing:562] tokens: [CLS] which nfl team represented the afc at super bowl 50 ? [SEP] super bowl 50 was an american football game to determine the champion of the national football league ( nfl ) for the 2015 season . the american football conference ( afc ) champion denver broncos defeated the national football conference ( nfc ) champion carolina panthers 24 – 10 to earn their third super bowl title . the game was played on february 7 , 2016 , at levi ' s stadium in the san francisco bay area at santa clara , california . as this was the 50th super bowl , the league emphasized the " golden anniversary

100%|██████████| 48/48 [00:06<00:00,  7.97it/s]

[NeMo I 2022-01-01 07:26:36 qa_dataset:119] loading from PATH_TO_DATA/squad/v1.1/dev-v1.1.json_cache_eval_BertTokenizer_30522_384_128_64_10000



[NeMo W 2022-01-01 07:26:37 nlp_overrides:174] Apex was not found. Using model parallel or megatron models will error out.
[NeMo W 2022-01-01 07:26:37 nlp_model:63] Apex was not found. Using model parallel or megatron models will error out.
[NeMo W 2022-01-01 07:26:37 modelPT:198] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a 

# TRAINING

In [ ]:
# start the training
trainer.fit(model)

[NeMo W 2022-01-01 07:27:13 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:292: LightningDeprecationWarning: Base `Callback.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      f"Base `Callback.{hook}` hook signature has changed in v1.5."
    
[NeMo W 2022-01-01 07:27:13 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:292: LightningDeprecationWarning: Base `Callback.on_train_batch_end` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      f"Base `Callback.{hook}` hook signature has changed in v1.5."
    
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2022-01-01 07:27:13 modelPT:471] The lightning trainer received accelerator: <pytorch_lightning.accelerators.gpu.GPUAccelerator object at 0x7f6474864c10>. We recommend to use 'ddp' instead.


[NeMo I 2022-01-01 07:27:13 modelPT:561] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        eps: 1e-08
        lr: 3e-05
        weight_decay: 0.0
    )
[NeMo I 2022-01-01 07:27:13 lr_scheduler:752] Scheduler "<nemo.core.optim.lr_scheduler.SquareRootAnnealing object at 0x7f64374fdfd0>" 
    will be used during training (effective maximum steps = 3384) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.0
    last_epoch: -1
    max_steps: 3384
    )



  | Name       | Type            | Params
-----------------------------------------------
0 | bert_model | BertEncoder     | 109 M 
1 | classifier | TokenClassifier | 1.5 K 
2 | loss       | SpanningLoss    | 0     
-----------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
218.968   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

[NeMo W 2022-01-01 07:27:17 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
      "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
    


...

[NeMo I 2022-01-01 10:20:55 qa_model:173] val exact match 68.79
[NeMo I 2022-01-01 10:20:55 qa_model:174] val f1 78.92813585301073


Epoch 2, global step 2537: val_loss reached 1.55325 (best 1.40243), saving model to "/content/PATH_TO_CHECKPOINTS_AND_LOGS/QA/2022-01-01_07-17-01/checkpoints/QA--val_loss=1.5532-epoch=2.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

[NeMo I 2022-01-01 11:18:53 qa_model:173] val exact match 68.88
[NeMo I 2022-01-01 11:18:53 qa_model:174] val f1 79.22517537820997


Epoch 3, global step 3383: val_loss was not in top 3
Saving latest checkpoint...


# TESTING IN DEV FILE

In [ ]:
model.setup_test_data(test_data_config=config.model.test_ds)
trainer.test(model)

100%|██████████| 48/48 [00:05<00:00,  9.52it/s]

[NeMo I 2022-01-01 11:20:36 qa_dataset:119] loading from PATH_TO_DATA/squad/v1.1/dev-v1.1.json_cache_eval_BertTokenizer_30522_384_128_64_10000



[NeMo W 2022-01-01 11:20:37 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:292: LightningDeprecationWarning: Base `Callback.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      f"Base `Callback.{hook}` hook signature has changed in v1.5."
    
[NeMo W 2022-01-01 11:20:37 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:292: LightningDeprecationWarning: Base `Callback.on_train_batch_end` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
      f"Base `Callback.{hook}` hook signature has changed in v1.5."
    
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[NeMo I 2022-01-01 11:36:44 qa_model:173] test exact match 68.88
[NeMo I 2022-01-01 11:36:44 qa_model:174] test f1 79.22517537820997
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_exact_match': 68.87999725341797,
 'test_f1': 79.22517395019531,
 'test_loss': 1.7902847528457642}
--------------------------------------------------------------------------------


[{'test_exact_match': 68.87999725341797,
  'test_f1': 79.22517395019531,
  'test_loss': 1.7902847528457642}]

In [ ]:
# # store test prediction under the experiment output folder
output_prediction_file = f"{exp_dir}/{INFER_PREDICTION_OUTPUT_FILE}"
output_nbest_file = f"{exp_dir}/{INFER_NBEST_OUTPUT_FILE}"
all_preds, all_nbests = model.inference(file=INFER_FILE, batch_size=5, num_samples=INFER_NUM_SAMPLES, output_nbest_file=output_nbest_file, output_prediction_file=output_prediction_file)

100%|██████████| 48/48 [00:05<00:00,  9.49it/s]


In [ ]:
for _, item in all_preds.items():
    print(f"question: {item[0]} answer: {item[1]}")
#The prediction file contains the predicted answer to each question id for the first TEST_NUM_SAMPLES.
!python -m json.tool $exp_dir/$INFER_PREDICTION_OUTPUT_FILE

question: Which NFL team represented the AFC at Super Bowl 50? answer: Denver Broncos
question: Which NFL team represented the NFC at Super Bowl 50? answer: Carolina Panthers
question: Where did Super Bowl 50 take place? answer: Levi's Stadium
question: Which NFL team won Super Bowl 50? answer: Denver Broncos
question: What color was used to emphasize the 50th anniversary of the Super Bowl? answer: gold
{
    "56be4db0acb8001400a502ec": [
        "Which NFL team represented the AFC at Super Bowl 50?",
        "Denver Broncos"
    ],
    "56be4db0acb8001400a502ed": [
        "Which NFL team represented the NFC at Super Bowl 50?",
        "Carolina Panthers"
    ],
    "56be4db0acb8001400a502ee": [
        "Where did Super Bowl 50 take place?",
        "Levi's Stadium"
    ],
    "56be4db0acb8001400a502ef": [
        "Which NFL team won Super Bowl 50?",
        "Denver Broncos"
    ],
    "56be4db0acb8001400a502f0": [
        "What color was used to emphasize the 50th anniversary of the 

In [ ]:
!cp "/content/PATH_TO_CHECKPOINTS_AND_LOGS/QA/2022-01-01_07-17-01/checkpoints/QA--val_loss=1.4432-epoch=1.ckpt" "/content/drive/MyDrive/NLP"

In [ ]:
!cp "/content/PATH_TO_CHECKPOINTS_AND_LOGS/QA/2022-01-01_07-17-01/output_prediction.json" "/content/drive/MyDrive/NLP"